En este notebook se carga el modelo desde Hugging Face y se lanza la interfaz para realizar las inferencias

🔹 Listado de artistas obtenidos del dataset de entrenamiento

In [1]:
artist_names = [
    'Andrés Calamaro',
    'Chaqueño Palavecino',
    'Fito Páez',
    'Horacio Guarany',
    'Jairo',
    'Juan Gabriel',
    'La Barra',
    'Leo Dan',
    'Litto Nebbia',
    'Los Chalchaleros',
    'Los Palmeras',
    'Mercedes Sosa',
    'Pablo Milanés',
    'Palito Ortega',
    'Raphael',
    'Rocío Dúrcal',
    'Sabroso',
    'Sandro',
    'Ulises Bueno',
    'Víctor Heredia'
]


## 🔹 Carga del modelo

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import os
import torch

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_NAME = "bernardoc90/gpt2-spanish-lyrics"

# Cargar Modelo desde hugging face
print(f"Cargando modelo original fine tuned: {MODEL_NAME}")
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.add_special_tokens({
    "pad_token": "<pad>",
    "bos_token": "<bos>",
    "eos_token": "<eos>"
})
tokenizer.add_tokens(["<art>", "<lyr>"])
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)
model.resize_token_embeddings(len(tokenizer))

# Mover el modelo al dispositivo
model = model.to(device)
print(f"Modelo en: {next(model.parameters()).device}")

# Verificar que los tokens especiales estén correctamente configurados:
print(f"Tokens especiales: {tokenizer.special_tokens_map}")
print(f"ID de <lyr>: {tokenizer.encode('<lyr>')}")
print(f"ID de <art>: {tokenizer.encode('<art>')}")
print(f"ID de <eos>: {tokenizer.encode('<eos>')}")

print(f"Tamaño vocabulario tokenizer: {len(tokenizer)}")
print(f"Tamaño embeddings modelo: {model.get_input_embeddings().weight.shape[0]}")

Cargando modelo original fine tuned: bernardoc90/gpt2-spanish-lyrics


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/750 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

Modelo en: cuda:0
Tokens especiales: {'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<|endoftext|>', 'pad_token': '<pad>', 'additional_special_tokens': ['<|talk|>', '<|ax1|>', '<|ax2|>', '<|ax3|>', '<|ax4|>', '<|ax5|>', '<|ax6|>', '<|ax7|>', '<|ax8|>', '<|ax9|>']}
ID de <lyr>: [50261]
ID de <art>: [50260]
ID de <eos>: [50259]
Tamaño vocabulario tokenizer: 50262
Tamaño embeddings modelo: 50262


## 🔹 Interfaz Gradio para realizar las inferencias

In [ ]:
import gradio as gr
import tempfile

# Parámetros de generación ahora fijos y definidos como constantes
FIXED_MAX_NEW_TOKENS = 512
FIXED_TEMPERATURE = 1.0
FIXED_TOP_K = 50
FIXED_TOP_P = 0.95

# Función para generar texto con parámetros personalizables
def generate_text(artist):
    max_new_tokens = FIXED_MAX_NEW_TOKENS
    temperature = FIXED_TEMPERATURE
    top_k = FIXED_TOP_K
    top_p = FIXED_TOP_P

    model.eval()
    inputs = tokenizer(f"<bos><art>{artist}<lyr>", return_tensors="pt")
    inputs = {key: value.to(device) for key, value in inputs.items()}
    outputs = model.generate(
        **inputs,
        max_new_tokens=int(max_new_tokens),  # Controla la cantidad de nuevos tokens generados
        temperature=float(temperature),      # Controla la aleatoriedad (solo si do_sample=True)
        top_k=int(top_k),                    # Top-k sampling
        top_p=float(top_p),                  # Top-p (nucleus) sampling
        do_sample=True,                      # Habilita muestreo (permite aleatoriedad - no determinismo)
        pad_token_id=tokenizer.pad_token_id, # Evita advertencias
        eos_token_id=tokenizer.eos_token_id, # Evita advertencias
    )

    text = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # Guardar en archivo temporal para descarga
    with tempfile.NamedTemporaryFile(delete=False, mode='w', suffix='.txt') as f:
        f.write(text)
        file_path = f.name

    return text, file_path

# Crear la interfaz de Gradio
interface = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Dropdown(choices=artist_names, label="Artista")
    ],
    theme="default",
    outputs=[
        gr.Textbox(label="Letra generada", lines=15, autoscroll=False),
        gr.File(label="Descargar letra")
    ],
    title=f"Generador de Letras de Canciones (DeepESP/gpt2-spanish)",
    description=f"Elegir un artista. Los parámetros de generación están pre-fijados a: Max Tokens={FIXED_MAX_NEW_TOKENS}, Temperature={FIXED_TEMPERATURE}, Top K={FIXED_TOP_K}, Top P={FIXED_TOP_P}. Usando {device.upper()}.",
    # Cambiar el texto del botón de submit
    submit_btn="🎵 Generar letra"
)

# Lanzar la interfaz
interface.launch(share=True)